In [ ]:
# install converter package
# see https://onnx.ai/sklearn-onnx/introduction.html

#!pip install --user skl2onnx

# after installation, need to restart the session and make sure to tick 'use python packages installed in CERNBox'

In [ ]:
# imports

import os
import sys
import json
import time
import joblib
import importlib
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

thisdir = os.getcwd()
topdir = os.path.abspath(os.path.join(thisdir, '../../../../'))
sys.path.append(topdir)

import tools.iotools as iotools
import tools.dftools as dftools
import plotting.plottools as plottools
from studies.clusters_2024.plotting.plot_cluster_occupancy import plot_cluster_occupancy
from studies.clusters_2024.nmf.nmf_testing_pattern import make_preprocessors
from studies.clusters_2024.nmf.nmf_testing_pattern import load_nmfs

### Load some previously stored models

In [ ]:
# set eras and layers for models to retrieve

eras = [
    'C-v1'
]

layers = [
    'BPix1',
    'BPix2',
    'BPix3',
    'BPix4'
]

# load nmf models

modeldir = '../output_20250604/models'

# set path
nmf_files = {}
for era in eras:
    nmf_files[era] = {}
    for layer in layers:
        nmf_files[era][layer] = os.path.join(modeldir, f'nmf_model_{layer.upper()}_{era}.pkl')
    
# existence check
missing = []
for era in eras:
    for layer, f in nmf_files[era].items():
        if not os.path.exists(f): missing.append(f)
if len(missing) > 0:
    raise Exception(f'The following files do not exist: {missing}')
    
# load models
nmfs = {}
for era in eras:
    nmfs[era] = {}
    for layer in layers:
        nmf_file = nmf_files[era][layer]
        nmf = joblib.load(nmf_file)
        nmfs[era][layer] = nmf

### Store and re-load models in ONNX format

In [ ]:
# store nmf models in ONNX format

# set eras and layers for models to store

eras = [
    'C-v1'
]

layers = [
    'BPix1',
    #'BPix2',
    #'BPix3',
    #'BPix4'
]

import nmf2d_onnx
importlib.reload(nmf2d_onnx)

# wrap models
from nmf2d_onnx import NMF2DTransformWrapper
nmfs_wrapped = {}
n_approx_steps = 1000
for era in eras:
    nmfs_wrapped[era] = {}
    for layer in layers:
        nmfs_wrapped[era][layer] = NMF2DTransformWrapper(nmfs[era][layer], n_approx_steps = n_approx_steps)

# register the converter
from skl2onnx import update_registered_converter
from nmf2d_onnx import skl2onnx_shape_calculator
from nmf2d_onnx import skl2onnx_converter
update_registered_converter(
    NMF2DTransformWrapper, "NMF2DTransformWrapper",
    skl2onnx_shape_calculator,
    skl2onnx_converter)

# get expected input shape
from skl2onnx.common.data_types import FloatTensorType
initial_types = {}
for layer in layers:
    shape = nmfs[eras[0]][layer].xshape
    initial_types[layer] = [
        ('input', FloatTensorType([None, *shape])),
    ]

# convert models to ONNX
from skl2onnx import convert_sklearn
for era in eras:
    for layer in layers:
        nmf_onnx = convert_sklearn(nmfs_wrapped[era][layer], initial_types=initial_types[layer])
        with open(f'test_{era}_{layer}.onnx', "wb") as f:
            f.write(nmf_onnx.SerializeToString())

In [ ]:
# read ONNX models

import onnxruntime as rt

nmfs_onnx = {}
for era in eras:
    nmfs_onnx[era] = {}
    for layer in layers:
        nmfs_onnx[era][layer] = rt.InferenceSession(f"test_{era}_{layer}.onnx", providers=["CPUExecutionProvider"])

### Read some data and compare predictions

In [ ]:
# set era and layers for evaluation

era = 'C-v1'
layers = [
    'BPix1',
    #'BPix2',
    #'BPix3',
    #'BPix4'
]

In [ ]:
# set path to input files

# settings
datadir = '/eos/user/l/llambrec/dialstools-output'
year = '2024'
dataset = 'ZeroBias'
reco = 'PromptReco'
mebase = 'PixelPhase1-Phase1_MechanicalView-PXBarrel-'
mebase += 'clusters_per_SignedModuleCoord_per_SignedLadderCoord_PXLayer_'
all_layers = ['BPix1', 'BPix2', 'BPix3', 'BPix4']

# find files corresponding to settings
input_files = {}
mainera, version = era.split('-', 1)
input_files[era] = {}
for layer in all_layers:
    f = f'{dataset}-Run{year}{mainera}-{reco}-{version}-DQMIO-{mebase}{layer[-1]}.parquet'
    f = os.path.join(datadir, f)
    input_files[era][layer] = f
    
# existence check
missing = []
present = []
for _, values in input_files.items():
    for layer, f in values.items():
        if not os.path.exists(f): missing.append(f)
        else: present.append(f)
if len(missing) > 0:
    raise Exception(f'The following files do not exist: {missing}')
else:
    print(f'Found {len(present)} files.')

In [ ]:
# make preprocessors

global_normalization = 'avg'
local_normalization = 'avg_era_C-v1'
preprocessors = make_preprocessors([era], all_layers,
                                   global_normalization = global_normalization,
                                   local_normalization = local_normalization)

In [ ]:
# load available run and lumisection numbers

dftemp = iotools.read_parquet(input_files[era][layers[0]], columns=['run_number', 'ls_number', 'entries'])
dftemp = dftemp[dftemp['entries']>0.5e6]
available_run_numbers = dftemp['run_number'].values
available_ls_numbers = dftemp['ls_number'].values
unique_runs = np.unique(available_run_numbers)

print('Available runs:')
print(unique_runs)

In [ ]:
# plot some random (or not random) examples

# random lumisections
nplot = 1
random_ids = np.random.choice(len(available_run_numbers), size=min(nplot, len(available_run_numbers)), replace=False)
selected_run_numbers = available_run_numbers[random_ids]
selected_ls_numbers = available_ls_numbers[random_ids]

# alternative: specific selected lumisections
#selected_runlumis = [(383486, 101), (383486, 102), (383486, 103)]
#selected_run_numbers = [el[0] for el in selected_runlumis]
#selected_ls_numbers = [el[1] for el in selected_runlumis]

if len(selected_run_numbers) > 0:
    
    # load data
    print('Loading data...')
    dfs = {}
    mes = {}
    for layer in layers:
        dfs[layer] = iotools.read_lumisections(input_files[era][layer], selected_run_numbers, selected_ls_numbers, mode='batched')
        mes[layer], runs, lumis = dftools.get_mes(dfs[layer], xbinscolumn='x_bin', ybinscolumn='y_bin', runcolumn='run_number', lumicolumn='ls_number')
    
    # preprocess
    print('Processing...')
    mes_preprocessed = {}
    mes_preprocessed_extra = {}
    for layer in layers:
        mes_preprocessed[layer] = preprocessors[era][layer].preprocess(dfs[layer])
        this_mes_preprocessed = np.copy(mes_preprocessed[layer])
        if preprocessors is not None:
            threshold = 5
            this_mes_preprocessed[this_mes_preprocessed > threshold] = threshold
        if preprocessors is not None:
            this_mes_preprocessed[this_mes_preprocessed == 0] = 1
        mes_preprocessed_extra[layer] = this_mes_preprocessed
    
    # predict direct
    print('Running raw inference...')
    mes_pred = {}
    for layer in layers:
        mes_pred[layer] = nmfs[era][layer].predict(mes_preprocessed_extra[layer])
        
    # predict
    print('Running ONNX inference...')
    mes_pred_onnx = {}
    for layer in layers:
        input_name = nmfs_onnx[era][layer].get_inputs()[0].name
        label_name = nmfs_onnx[era][layer].get_outputs()[0].name
        mes_pred_onnx[layer] = nmfs_onnx[era][layer].run([label_name], {input_name: mes_preprocessed_extra[layer].astype(np.float32)})[0]
        
    # make the plots
    print('Plotting...')
    for idx in range(len(selected_run_numbers)):
        run = runs[idx]
        lumi = lumis[idx]
        for layer in layers:
            me_orig = mes[layer][idx, :, :]
            me_preprocessed = mes_preprocessed[layer][idx, :, :]
            me_pred = mes_pred[layer][idx, :, :]
            me_pred_onnx = mes_pred_onnx[layer][idx, :, :]
    
            # initialize figure
            nrows = 1
            figheight = 6
            fig, axs = plt.subplots(ncols=4, nrows=nrows, figsize=(24, figheight), squeeze=False)
            
            # plot raw data
            fig, axs[0, 0] = plot_cluster_occupancy(me_orig, fig=fig, ax=axs[0, 0],
                   title='Raw', titlesize=15,
                   xaxtitlesize=15, yaxtitlesize=15,
                   ticklabelsize=12, colorticklabelsize=12,
                   docolorbar=True, caxtitle='Number of clusters',
                   caxtitlesize=15, caxtitleoffset=15)
        
            # plot preprocessed, reconstructed and ONNX reconstructed
            fig, axs[0, 1] = plot_cluster_occupancy(me_preprocessed, fig=fig, ax=axs[0, 1],
                   title='Input', titlesize=15,
                   xaxtitlesize=15, yaxtitlesize=15,
                   ticklabelsize=12, colorticklabelsize=12,
                   docolorbar=True, caxtitle='Number of clusters\n(normalized)',
                   caxrange=(1e-6,2),
                   caxtitlesize=15, caxtitleoffset=30)
            fig, axs[0, 2] = plot_cluster_occupancy(me_pred, fig=fig, ax=axs[0, 2],
                   title='Reconstructed', titlesize=15,
                   xaxtitlesize=15, yaxtitlesize=15,
                   ticklabelsize=12, colorticklabelsize=12,
                   docolorbar=True, caxtitle='Number of clusters\n(normalized)',
                   caxrange=(1e-6,2),
                   caxtitlesize=15, caxtitleoffset=30)
            fig, axs[0, 3] = plot_cluster_occupancy(me_pred_onnx, fig=fig, ax=axs[0, 3],
                   title='ONNX reconstructed', titlesize=15,
                   xaxtitlesize=15, yaxtitlesize=15,
                   ticklabelsize=12, colorticklabelsize=12,
                   docolorbar=True, caxtitle='Number of clusters\n(normalized)',
                   caxrange=(1e-6,2),
                   caxtitlesize=15, caxtitleoffset=30)
                
            # plot aesthetics
            plt.subplots_adjust(wspace=0.55)
            if str(layer)=='BPix1': plt.subplots_adjust(hspace=-0.65)
            if str(layer)=='BPix2': plt.subplots_adjust(hspace=-0.35)
            title = f'Run {run}, LS {lumi}, layer {layer}'
            axs[0, 0].text(0.01, 1.3, title, fontsize=15, transform=axs[0, 0].transAxes)
            plt.show()
            plt.close()
            
            # plot the difference between both reconstructions
            me_pred_diff = np.abs(me_pred - me_pred_onnx)
            fig, ax = plt.subplots()
            fig, ax = plot_cluster_occupancy(me_pred_diff, fig=fig, ax=ax,
                   title='Difference between raw and ONNX reconstruction', titlesize=15,
                   xaxtitlesize=15, yaxtitlesize=15,
                   ticklabelsize=12, colorticklabelsize=12,
                   docolorbar=True, caxtitle='Difference',
                   caxrange=(0, 0.01),
                   caxtitlesize=15, caxtitleoffset=15)
            title = f'Run {run}, LS {lumi}'
            ax.text(0.01, 1.3, title, fontsize=15, transform=ax.transAxes)
            plt.show()
            plt.close()

### Read a larger batch of data and compare runtimes

In [ ]:
# set era and layers for evaluation

era = 'C-v1'
layers = [
    'BPix1',
    #'BPix2',
    #'BPix3',
    #'BPix4'
]

In [ ]:
import time

# read data
for layer in layers:
    batch_size = int(5000 / int(layer[-1]))
    print(f'Running on layer {layer}')
    
    # load the data
    print('  Loading data...')
    df = iotools.read_parquet(input_files[era][layer], verbose=False, batch_size=batch_size, first_batch=0, last_batch=0)
    df = df[df['entries']>0]
    
    # preprocess
    print('Processing...')
    mes_preprocessed = preprocessors[era][layer].preprocess(df)
    if preprocessors is not None:
        threshold = 5
        mes_preprocessed[mes_preprocessed > threshold] = threshold
    if preprocessors is not None:
        mes_preprocessed[mes_preprocessed == 0] = 1

    # predict direct
    print('Running raw inference...')
    starttime = time.time()
    mes_pred = nmfs[era][layer].predict(mes_preprocessed)
    stoptime = time.time()
    timing_raw = stoptime - starttime
    timing_raw_avg = timing_raw / len(df)
        
    # predict
    print('Running ONNX inference...')
    starttime = time.time()
    input_name = nmfs_onnx[era][layer].get_inputs()[0].name
    label_name = nmfs_onnx[era][layer].get_outputs()[0].name
    mes_pred_onnx = nmfs_onnx[era][layer].run([label_name], {input_name: mes_preprocessed.astype(np.float32)})[0]
    stoptime = time.time()
    timing_onnx = stoptime - starttime
    timing_onnx_avg = timing_onnx / len(df)
    
    # printouts
    print(f'=== Timing for layer {layer} ===')
    print(f'  - on {len(df)} instances:' + ' raw: {:.3f} s,  ONNX: {:.3f} s'.format(timing_raw, timing_onnx))
    print(f'  - avg per instances:' + ' raw: {:.3f} ms,  ONNX: {:.3f} ms'.format(timing_raw_avg*1000, timing_onnx_avg*1000))

### Plot model saving, loading, and evaluation time as a function of number of approximation steps

In [ ]:
# get the data

# settings
era = 'C-v1'
layers = [
    'BPix1',
    'BPix2',
    'BPix3',
    'BPix4'
]
batch_size = 5000
points = [10, 20, 50, 100, 200, 500, 1000]
nrepeats = 10

import nmf2d_onnx
importlib.reload(nmf2d_onnx)
from nmf2d_onnx import NMF2DTransformWrapper
from skl2onnx import update_registered_converter
from nmf2d_onnx import skl2onnx_shape_calculator
from nmf2d_onnx import skl2onnx_converter
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnxruntime as rt

# initializations
store_times = {}
store_sizes = {}
model_sizes = {}
load_times = {}
run_times = {}
accuracy = {}

# register the converter
update_registered_converter(
        NMF2DTransformWrapper, "NMF2DTransformWrapper",
        skl2onnx_shape_calculator,
        skl2onnx_converter)

# get expected input shape
initial_types = {}
for layer in layers:
    shape = nmfs[eras[0]][layer].xshape
    initial_types[layer] = [
        ('input', FloatTensorType([None, *shape])),
    ]

# loop over layers
for layer in layers:
    store_times[layer] = {}
    store_sizes[layer] = {}
    model_sizes[layer] = {}
    load_times[layer] = {}
    run_times[layer] = {}
    accuracy[layer] = {}
    
    # load data
    print(f'Preparing data for {layer}...')
    df = iotools.read_parquet(input_files[era][layer], verbose=False, batch_size=batch_size, first_batch=0, last_batch=0)
    df = df[df['entries']>0]
    mes_preprocessed = preprocessors[era][layer].preprocess(df)
    if preprocessors is not None:
        threshold = 5
        mes_preprocessed[mes_preprocessed > threshold] = threshold
    if preprocessors is not None:
        mes_preprocessed[mes_preprocessed == 0] = 1
        
    # make raw reconstruction
    mes_pred = nmfs[era][layer].predict(mes_preprocessed)
    
    # loop over points
    for point in points:
        print(f'Running on layer {layer}, point {point}...')
        store_times[layer][point] = []
        store_sizes[layer][point] = []
        model_sizes[layer][point] = []
        load_times[layer][point] = []
        run_times[layer][point] = []
        accuracy[layer][point] = []
        
        # loop over repeats
        for _ in range(nrepeats):
        
            # store model in ONNX
            start_time = time.time()
            nmf_wrapped = NMF2DTransformWrapper(nmfs[era][layer], n_approx_steps = point)
            nmf_onnx = convert_sklearn(nmf_wrapped, initial_types=initial_types[layer])
            with open(f'test_{layer}.onnx', "wb") as f:
                f.write(nmf_onnx.SerializeToString())
            stop_time = time.time()
            store_times[layer][point].append(stop_time - start_time)
        
            # find model size
            store_sizes[layer][point].append(os.path.getsize(f'test_{layer}.onnx') / 1024)
        
            # load model from ONNX
            start_time = time.time()
            nmf_onnx = rt.InferenceSession(f"test_{layer}.onnx", providers=["CPUExecutionProvider"])
            stop_time = time.time()
            load_times[layer][point].append(stop_time - start_time)
            model_sizes[layer][point].append(sys.getsizeof(nmf_onnx) / 1024)
            # (note: not clear if sys.getsizeof is accurate, it might return just the size of some pointers,
            #        not the complete size of the total object.)
        
            # run inference
            start_time = time.time()
            input_name = nmf_onnx.get_inputs()[0].name
            label_name = nmf_onnx.get_outputs()[0].name
            mes_pred_onnx = nmf_onnx.run([label_name], {input_name: mes_preprocessed.astype(np.float32)})[0]
            stop_time = time.time()
            timing = (stop_time - start_time) / len(df)
            run_times[layer][point].append(timing)
            
            # calculate average accuracy per bin per monitoring element
            numerator = np.sum(np.abs(mes_pred_onnx - mes_pred))
            denominator = len(df) * mes_pred.shape[1] * mes_pred.shape[2]
            acc = numerator / denominator
            accuracy[layer][point].append(acc)

In [ ]:
# plot the data

def plot(info, yaxtitle='Time (s)', yaxlog=False):

    layers = sorted(list(info.keys()))
    points = sorted(list(info[layers[0]].keys()))
    
    colordict = {
        'BPix1': 'navy',
        'BPix2': 'royalblue',
        'BPix3': 'mediumslateblue',
        'BPix4': 'darkorchid'
    }
    
    fig, ax = plt.subplots()
    for layer in layers:
        vals = [info[layer][point] for point in points]
        # remove clear outliers
        medians = [np.median(v) for v in vals]
        iqrs = [(np.quantile(v, 0.75) - np.quantile(v, 0.25)) for v in vals]
        for idx, v in enumerate(vals):
            vals[idx] = [n for n in v if (n>=medians[idx]-2*iqrs[idx] and n<=medians[idx]+2*iqrs[idx])]
        avgs = [np.mean(v) for v in vals]
        errs = [np.std(v) for v in vals]
        ax.errorbar(points, avgs, yerr=errs, c=colordict[layer], fmt='.')
        ax.plot(points, avgs, color=colordict[layer], label=layer, linestyle='--')

    ax.set_xscale('log')
    if yaxlog: ax.set_yscale('log')
        
    ax.set_xlabel('Number of approximation steps', fontsize=12)
    ax.set_ylabel(yaxtitle, fontsize=12)
    ax.grid()
    ax.legend(fontsize=12)
    
    return fig, ax
    
fig, ax = plot(store_times)
ax.text(0, 1.02, 'Model storing time', fontsize=12, transform=ax.transAxes)

fig, ax = plot(store_sizes, yaxtitle='Size (kB)')
ax.text(0, 1.02, 'Model size on disk', fontsize=12, transform=ax.transAxes)

fig, ax = plot(load_times)
ax.text(0, 1.02, 'Model loading time', fontsize=12, transform=ax.transAxes)

fig, ax = plot(model_sizes, yaxtitle='Size (kB)')
ax.text(0, 1.02, 'Model size in memory', fontsize=12, transform=ax.transAxes)

fig, ax = plot(run_times)
ax.text(0, 1.02, 'Inference time (per instance)', fontsize=12, transform=ax.transAxes)

fig, ax = plot(accuracy, yaxtitle='Average per-bin residual', yaxlog=True)
ax.text(0, 1.02, 'Model accuracy', fontsize=12, transform=ax.transAxes)